## Colab specific cells

In [ ]:
!apt update
!pip install pyvirtualdisplay
!apt-get install -y xvfb python-opengl ffmpeg
!pip install gym[atari,box2d,classic_control]

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# root_path = 'gdrive/My Drive/Colab Notebooks/RL/'
# import os
# os.chdir(root_path)

In [ ]:
# %tensorflow_version 2.x
# %tensorflow_version 1.x


In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

In [ ]:
import glob
import io
from IPython.display import HTML
from IPython import display as ipythondisplay
import base64

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[-1]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)

        # you can add "loop" after autoplay to keep the video looping after it ends
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                     controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                 </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

        

from gym.wrappers import Monitor

def wrap_env(env):
    env = Monitor(env, './video', force=True)
    return env

## If not using colab change the value to false

In [ ]:
using_colab = False

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if "cuda" in str(device):
    print("Using GPU")
else:
    print("Not using GPU")

## Policy Gradient Algorithm (REINFORCE)

In [ ]:
import gym
from gym import logger as gymlogger
gymlogger.set_level(40) #error only

import numpy as np
import random
import math

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
env_list = ["LunarLander-v2", "MsPacman-ram-v0", "CartPole-v0", "MountainCar-v0",
            "Breakout-ram-v4", "Acrobot-v1"]

env_to_use = env_list[2]

### Selecting the environment and exploring it

In [ ]:
env = gym.make(env_to_use)
if using_colab:
    env = wrap_env(env)

#check out the environment's action and observation space!
print(env.action_space)
print(env.observation_space)

from gym import spaces
assert type(env.observation_space)==spaces.Box, print("State space should be continuous")
assert len(env.observation_space.shape)==1, print("State space should be 1-D")
assert type(env.action_space)==spaces.Discrete, print("Action space should be discrete")

observation = env.reset()
while True:
    env.render()
    #your agent goes here
    action = env.action_space.sample() # selecting a random action from the action space
    observation, reward, done, info = env.step(action) 
    if done: 
        break
    # break

env.close()
if using_colab:
    show_video()


### Defining the Network to be used

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PolicyNet(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(PolicyNet, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

class PGLoss(nn.Module):
    def forward(self, policy, act, rew_wt):
        logp = policy.log_prob(act)
        return -(logp * rew_wt).mean()


### Defining the agent

In [ ]:
import torch
import torch.optim as optim

from torch.distributions.categorical import Categorical

from collections import defaultdict

class Agent:
    def __init__(self, env, batch_size, device='cpu'):
        self.env = env
        self.device = device
        
        self.state_size = self.env.observation_space.shape[0]
        self.action_size = self.env.action_space.n

        self.policy_network = PolicyNet(self.state_size, self.action_size, 4).to(device)
        self.loss = PGLoss()
        self.batch_size = batch_size
        self.train_stats = defaultdict(list)

    def get_policy(self, obs):
        net_op = self.policy_network(torch.as_tensor(obs, dtype=torch.float32).to(self.device))
        return Categorical(logits=net_op.cpu())
    def get_action(self, obs):
        return self.get_policy(obs).sample().item()

    def get_episode_batch(self):
        batch_obs = []
        batch_action = []
        batch_weights = []
        batch_returns = []
        batch_lengths = []

        cur_obs = self.env.reset()
        episode_rewards = []
        done = False

        while True:
            batch_obs.append(cur_obs.copy())
            
            cur_action = self.get_action(cur_obs)
            cur_obs, cur_reward, done, _ = self.env.step(cur_action)

            batch_action.append(cur_action)
            episode_rewards.append(cur_reward)
            
            if done:
                episode_return, episode_length = sum(episode_rewards), len(episode_rewards)
                
                batch_returns.append(episode_return)
                batch_lengths.append(episode_length)
                
                def get_reward_to_go(rewards_list):
                    rtg = []
                    sum_rewards = 0
                    for i in rewards_list[::-1]:
                        sum_rewards+=i
                        rtg.append(sum_rewards)
                    return rtg[::-1]
                batch_weights += get_reward_to_go(episode_rewards)
                
                if len(batch_obs) > self.batch_size:
                    break
                
                cur_obs, done, episode_rewards = self.env.reset(), False, []
        return batch_obs, batch_action, batch_weights, batch_returns, batch_lengths

    def train(self, epochs):
        optimizer = optim.Adam(self.policy_network.parameters(), lr=1e-2)
        for i in range(epochs):
            obs, act, rew_wt, eps_rew, eps_len = self.get_episode_batch()
            obs = torch.as_tensor(obs)
            act = torch.as_tensor(act)
            rew_wt = torch.as_tensor(rew_wt)
            
            optimizer.zero_grad()
            loss = self.loss(self.get_policy(obs), act, rew_wt)
            loss.backward()
            optimizer.step()

            self.train_stats["total_loss"] += [loss.item()]
            self.train_stats["rewards"] += [np.mean(eps_rew)]

            print("Epoch:", i, loss.item(), np.mean(eps_rew), np.mean(eps_len))
    
    def plot_train_stats(self):
        if len(self.train_stats)==0:
            print("first train to print train stats")
        for i in self.train_stats:
            plt.plot(self.train_stats[i])
            plt.xlabel("Epoch")
            plt.ylabel(i)
            plt.show()
        return


### Creating Agent's instance and using it to train

In [ ]:
print("GPU available:", torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

env = gym.make(env_to_use)
if using_colab:
    env = wrap_env(env)

agent = Agent(env, batch_size=5000, device=device)
agent.train(100)


In [ ]:
agent.plot_train_stats()

In [ ]:
# watch the trained agent
env = gym.make(env_to_use)
if using_colab:
    env = wrap_env(env)
state = env.reset()
done=False
while not done:
    action = agent.get_action(state)
    env.render()
    state, reward, done, _ = env.step(action)
    if done:
        break
        
env.close()
if using_colab:
    show_video()